# Ontology Mapping Demo

This notebook demonstrates the ontology mapping process using different strategies.

In [1]:
%cd '/Users/abhilashdhal/Desktop/cbioportal_gsoc/OmicsMLRepoHarmonizer'

/Users/abhilashdhal/Desktop/cbioportal_gsoc/OmicsMLRepoHarmonizer


In [1]:
%cd ..

/home/lcc/projects/MetaHarmonizer


In [2]:
import gevent.monkey
gevent.monkey.patch_all(thread=False, select=False)

True

In [3]:
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
import os
import pandas as pd
from importlib import reload
import src.models.ontology_mapper_st as om_st
from src.Engine import ontology_mapping_engine as ome
import src.models.ontology_mapper_lm as om_lm
reload(om_st)
reload(ome)
reload(om_lm)

/home/lcc/miniconda3/envs/gsoc/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lcc/miniconda3/envs/gsoc/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to /home/lcc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lcc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/lcc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/lcc/nltk_data...
[nltk_data]   Package s

<module 'src.models.ontology_mapper_lm' from '/home/lcc/projects/MetaHarmonizer/src/models/ontology_mapper_lm.py'>

## Load the data

In [4]:
cura_map = pd.read_csv('./data/corpus/cbio_treatment_name/trt_name_query_for_NCIT:C1909.csv')
query_list = cura_map['original_value'].tolist()
small_corpus_list = cura_map['curated_ontology'].to_list()
# large_corpus_list = pd.read_csv('./data/corpus_from_NCIT_C1909.csv')

In [5]:
cura_map 

,original_value,curated_ontology,curated_ontology_term_id,curated_ontology_term_db
0,17-DMAG,Alvespimycin,NCIT:C38142,NCIT
1,19D12,Robatumumab,NCIT:C71527,NCIT
2,5 fu,Fluorouracil,NCIT:C505,NCIT
3,5-FU,Fluorouracil,NCIT:C505,NCIT
4,5-fluorouracil,Fluorouracil,NCIT:C505,NCIT
...,...,...,...,...
573,vinblastine,Vinblastine,NCIT:C930,NCIT
574,vincrisinte,Vincristine,NCIT:C933,NCIT
575,vincristine,Vincristine,NCIT:C933,NCIT
576,vincristine sulfate,Vincristine Sulfate,NCIT:C1739,NCIT


## Initialize the Engine

In [6]:
other_params = {"test_or_prod": "prod"}
onto_engine_large = ome.OntoMapEngine(method='sap-bert', topk=5,
query=query_list, corpus=small_corpus_list,
cura_map=None, 
yaml_path='./src/models/method_model.yaml', 
om_strategy='lm', **other_params)

21/05//2025 12:47:1747856869 PM - INFO - OntoMapEngine: Initialized OntoMap Engine module


## Run the Ontology Mapping

In [7]:
results_engine_testing = onto_engine_large.run()

21/05//2025 12:47:1747856872 PM - INFO - OntoMapEngine: Running Ontology Mapping
21/05//2025 12:47:1747856872 PM - INFO - OntoMapEngine: Separating exact and non-exact matches
21/05//2025 12:47:1747856872 PM - INFO - OntoMapEngine: Running OntoMap model for non-exact matches
21/05//2025 12:47:1747856872 PM - INFO - OntoMapLM: Initialized OntoMap Language Model module
21/05//2025 12:47:1747856872 PM - INFO - OntoMapLM.get_match_results: Creating embeddings for query_list and corpus
21/05//2025 12:48:1747856895 PM - INFO - OntoMapLM.get_match_results: Calculating cosine similarity matrix
21/05//2025 12:48:1747856895 PM - INFO - OntoMapLM.get_match_results: Generating results table
21/05//2025 12:48:1747856896 PM - INFO - OntoMapLM.get_match_results: Results Generated


## Display Results

In [8]:
results_engine_testing

,original_value,curated_ontology,match_level,stage,top1_match,top1_score,top2_match,top2_score,top3_match,top3_score,top4_match,top4_score,top5_match,top5_score
0,Abiraterone,Abiraterone,1,1,Abiraterone,1.0,Abiraterone,1.0,Abiraterone,1.0,Abiraterone,1.0,Abiraterone,1.0
1,Adalimumab,Adalimumab,1,1,Adalimumab,1.0,Adalimumab,1.0,Adalimumab,1.0,Adalimumab,1.0,Adalimumab,1.0
2,Afatinib,Afatinib,1,1,Afatinib,1.0,Afatinib,1.0,Afatinib,1.0,Afatinib,1.0,Afatinib,1.0
3,Alisertib,Alisertib,1,1,Alisertib,1.0,Alisertib,1.0,Alisertib,1.0,Alisertib,1.0,Alisertib,1.0
4,Alpelisib,Alpelisib,1,1,Alpelisib,1.0,Alpelisib,1.0,Alpelisib,1.0,Alpelisib,1.0,Alpelisib,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,vinblastine,Not Available for Prod Environment,99,2,Vinblastine,1.0000,Vinblastine,1.0000,Vinblastine,1.0000,Vinorelbine,0.7998,Vinorelbine,0.7998
574,vincrisinte,Not Available for Prod Environment,99,2,Vincristine,0.8570,Vincristine,0.8570,Vincristine,0.8570,Vincristine,0.8570,Vincristine,0.8570
575,vincristine,Not Available for Prod Environment,99,2,Vincristine,1.0000,Vincristine,1.0000,Vincristine,1.0000,Vincristine,1.0000,Vincristine,1.0000
576,vincristine sulfate,Not Available for Prod Environment,99,2,Vincristine Sulfate,1.0000,Vincristine Sulfate,1.0000,Vincristine,0.8704,Vincristine,0.8704,Vincristine,0.8704


In [9]:
df = results_engine_testing

# total count of records
total = len(df)

# unmatched（match_level == 99）
unmatched_count = (df['match_level'] == 99).sum()
unmatched_ratio = unmatched_count / total

# matched（match_level != 99）
matched_df = df[df['match_level'] != 99]
matched_count = len(matched_df)
average_match_level = matched_df['match_level'].mean()

print(f"Total records: {total}")
print(f"Unmatched records: {unmatched_count} ({unmatched_ratio:.2%})")
print(f"Matched records: {matched_count} ({average_match_level:.2f})")

Total records: 578
Unmatched records: 436 (75.43%)
Matched records: 142 (1.00)


In [10]:
results_engine_testing.to_csv('results_engine_testing.csv', index=False)